In [ ]:
import os
import json

import torch
import tqdm

from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_data = json.load(open("/content/drive/MyDrive/Task-2-SemEval-2024-main/Task-2-SemEval-2024-main/training_data/train.json"))
test_data = json.load(open("/content/drive/MyDrive/Task-2-SemEval-2024-main/Task-2-SemEval-2024-main/test.json"))
files = os.listdir("/content/drive/MyDrive/Task-2-SemEval-2024-main/Task-2-SemEval-2024-main/training_data/CT json")
files.remove(".DS_Store")

In [ ]:
#Create a dictionary where keys are derived from filenames
files_data = {file[:-5]:json.load(open(f"/content/drive/MyDrive/Task-2-SemEval-2024-main/Task-2-SemEval-2024-main/training_data/CT json/{file}")) for file in files}

In [ ]:
# Define the training function
def train(model, train_data, tokenizer, device):
    model.train()
    total_loss = 0
    for batch in train_data:
        inputs = tokenizer(batch['premise'], batch['hypothesis'], truncation=True, padding=True, return_tensors='pt').to(device)
        outputs = model(**inputs, labels=inputs['input_ids']).loss
        total_loss += outputs.item()
    return total_loss / len(train_data)

In [ ]:
# Define the evaluation function
def evaluate(model, test_data, tokenizer, device):
    model.eval()
    predictions = []
    for batch in test_data:
        inputs = tokenizer(batch['premise'], batch['hypothesis'], truncation=True, padding=True, return_tensors='pt').to(device)
        outputs = model.generate(inputs['input_ids'], num_beams=4, max_length=15, early_stopping=True)
        predictions.append({'id': batch['id'], 'prediction': tokenizer.decode(outputs[0], skip_special_tokens=True)})
    return predictions

In [ ]:
# Define the evaluation function
data_expanded = []
for _id, value in test_data.items():
    temp = {}
    temp["id"] = _id
    p_nctid = value["Primary_id"]     #extract values from data
    s_nctid = value.get("Secondary_id")
    section_id = value["Section_id"]
    statement = value["Statement"]
    primary_evidence = files_data[p_nctid][section_id]
    temp["statement"] = statement
    temp["primary_evidence"] = primary_evidence
    # temp["label"] = value["Label"]

    if s_nctid is not None:
        secondary_evidence = files_data[s_nctid][section_id]
        temp["secondary_evidence"] = secondary_evidence

    data_expanded.append(temp)

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained('michiyasunaga/BioLinkBERT-base')
model = AutoModel.from_pretrained('michiyasunaga/BioLinkBERT-base')
inputs = tokenizer("Sunitinib is a tyrosine kinase inhibitor", return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state

In [ ]:
def get_input_text(premise, hypothesis):
    options_prefix = "options: "
    separator = ", "
    options_ = options_prefix + separator.join(["entailment", "contradiction"])
    return f"{premise} [SEP] {hypothesis} [SEP] {options_}"

In [ ]:
def train(model, train_data, tokenizer, device):
    model.train()
    total_loss = 0
    for batch in train_data:
        inputs = tokenizer(batch['premise'], batch['hypothesis'], truncation=True, padding=True, return_tensors='pt').to(device)
        labels = torch.tensor(batch['label']).to(device)
        outputs = model(**inputs, labels=labels)
        total_loss += outputs.loss.item()
    return total_loss / len(train_data)

def evaluate(model, test_data, tokenizer, device):
    model.eval()
    predictions = []
    for batch in test_data:
        inputs = tokenizer(batch['premise'], batch['hypothesis'], truncation=True, padding=True, return_tensors='pt').to(device)
        outputs = model.generate(inputs['input_ids'], num_beams=4, max_length=15, early_stopping=True)
        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append({'id': batch['id'], 'prediction': prediction})
    return predictions

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load pre-trained bioLinkBERT model and tokenizer
model_name = "michiyasunaga/BioLinkBERT-base"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load train data
train_data = json.load(open("/content/drive/MyDrive/Task-2-SemEval-2024-main/Task-2-SemEval-2024-main/training_data/train.json"))
# Train the model
total_loss = train(model, train_data, tokenizer, device)
print(f"Average training loss: {total_loss:.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at michiyasunaga/BioLinkBERT-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: string indices must be integers

In [ ]:
pred


['Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Contradiction',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Entailment',
 'Contradiction',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment',
 'Entailment'

In [ ]:
from collections import Counter

# Example Counter result
counter_result = Counter({1: 4726, 0: 774})
# Mapping for Counter values to categories
category_mapping = {0: "Entailment", 1: "Contradiction"}

# Convert Counter values to categories
category_result = {category_mapping[key]: value for key, value in counter_result.items()}
print(category_result)

{'Contradiction': 4726, 'Entailment': 774}


In [ ]:
#pred = [p[5:][:-4].strip() for p in pred]

In [ ]:
#set(pred)
#from collections import Counter
#Counter(pred)

Counter({'l': 4726, 'adic': 774})

In [ ]:
prediction_dict = {}
for _id,pred_x in zip(test_data, pred):
    prediction_dict[str(_id)] = {"Prediction":pred_x}

In [ ]:
from sklearn.metrics import f1_score
uuid_list = list(prediction_dict.keys())
results_pred = []
gold_labels = []
for i in range(len(uuid_list)):
     if prediction_dict[uuid_list[i]]["Prediction"] in ["Entailment", "Yes"]:
         results_pred.append(1)
     else:
         results_pred.append(0)
     if test_data[uuid_list[i]]["Type"] in ["Entailment", "No"]:
         gold_labels.append(1)
     else:
         gold_labels.append(0)
f1_score(gold_labels,results_pred)

0.0

In [ ]:
prediction_dict
json.dump(prediction_dict, open("ayesha1.json", 'w'), indent=4)


In [ ]:
#pred = [p[5:][:-4].strip() for p in pred]
#print(Counter(pred))
#prediction_dict = {}

#for _id, pred_x in zip(test_data, pred):
#    if pred_x.lower() == "no":
 #       pred_x = "Contradiction"
  #  elif pred_x.lower() == "yes":
   #     pred_x = "Entailment"
    # else:
    #     print(pred_x)
   # prediction_dict[str(_id)] = {"Prediction": pred_x}

#json.dump(prediction_dict, open("results.json", 'w'), indent=4)
#!zip "small_large/results_{model_name}.zip" results.json